In [631]:
import pandas as pd
import numpy as np

from tqdm import tqdm
from datetime import datetime as dt

'''
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.cluster import KMeans
from yellowbrick.cluster import InterclusterDistance
'''

'\nimport seaborn as sns\nfrom mpl_toolkits.mplot3d import Axes3D\nfrom scipy.cluster.hierarchy import dendrogram, linkage\nfrom sklearn.cluster import KMeans\nfrom yellowbrick.cluster import InterclusterDistance\n'

In [506]:
data = pd.read_csv('bank_transactions.csv', delimiter = ',')

In [500]:
data.columns

Index(['TransactionID', 'CustomerID', 'CustomerDOB', 'CustGender',
       'CustLocation', 'CustAccountBalance', 'TransactionDate',
       'TransactionTime', 'TransactionAmount (INR)'],
      dtype='object')

In [ ]:
# cleaning data
# dropping na, reformatting date times

#
data.dropna(inplace=True)#? doesnt change
data.CustomerDOB = pd.to_datetime(data.CustomerDOB) 
data.TransactionDate = pd.to_datetime(data.TransactionDate) 
data.TransactionTime = data.TransactionTime.apply(lambda x: dt.strptime(str(x).zfill(6), '%H%M%S').time() )

In [511]:
data.loc[data.CustomerDOB.dt.year > 2000, 'CustomerDOB'] = data.loc[data.CustomerDOB.dt.year > 2000, 'CustomerDOB'] - pd.DateOffset(years = 100)
data.drop(data[data.CustomerDOB.dt.year < 1900].index, axis=0, inplace=True)

In [512]:
data

,TransactionID,CustomerID,CustomerDOB,CustGender,CustLocation,CustAccountBalance,TransactionDate,TransactionTime,TransactionAmount (INR)
0,T1,C5841053,1994-10-01,F,JAMSHEDPUR,17819.05,2016-02-08,14:32:07,25.0
1,T2,C2142763,1957-04-04,M,JHAJJAR,2270.69,2016-02-08,14:18:58,27999.0
2,T3,C4417068,1996-11-26,F,MUMBAI,17874.44,2016-02-08,14:27:12,459.0
3,T4,C5342380,1973-09-14,F,MUMBAI,866503.21,2016-02-08,14:27:14,2060.0
4,T5,C9031234,1988-03-24,F,NAVI MUMBAI,6714.43,2016-02-08,18:11:56,1762.5
...,...,...,...,...,...,...,...,...,...
1048562,T1048563,C8020229,1990-08-04,M,NEW DELHI,7635.19,2016-09-18,18:48:24,799.0
1048563,T1048564,C6459278,1992-02-20,M,NASHIK,27311.42,2016-09-18,18:37:34,460.0
1048564,T1048565,C6412354,1989-05-18,M,HYDERABAD,221757.06,2016-09-18,18:33:13,770.0
1048565,T1048566,C6420483,1978-08-30,M,VISAKHAPATNAM,10117.87,2016-09-18,18:47:06,1000.0


#  Getting your data + feature engineering

In [523]:
data.columns

Index(['TransactionID', 'CustomerID', 'CustomerDOB', 'CustGender',
       'CustLocation', 'CustAccountBalance', 'TransactionDate',
       'TransactionTime', 'TransactionAmount (INR)'],
      dtype='object')

In [605]:
# nr_trans
nr_trans = data[['CustomerID','TransactionID']].groupby(by = 'CustomerID').count()
nr_trans.rename(columns = {'TransactionID' : 'nr_trans'}, inplace = True)

In [657]:
# nr_trans_100p
nr_trans_100p = data[['CustomerID','TransactionAmount (INR)']].groupby(by = 'CustomerID')
nr_trans_100p = nr_trans_100p.apply(lambda x: x[x['TransactionAmount (INR)']>100]['TransactionAmount (INR)'].count())

In [680]:
nr_trans_100p = pd.DataFrame(nr_trans_100p).set_index('CustomerID')
nr_trans_100p.columns = ['nr_trans_100p']

In [589]:
#  mean_trans 
mean_trans = data[['CustomerID','TransactionAmount (INR)']].groupby(by='CustomerID').mean()
mean_trans.rename(columns = {'TransactionAmount (INR)':'mean_trans' }, inplace = True)

In [592]:
# mean_balance
mean_balance = data[['CustomerID','CustAccountBalance']].groupby(by='CustomerID').mean()
mean_balance.rename(columns = {'CustAccountBalance':'mean_balance' }, inplace = True)

In [603]:
# mean_diff
#mean_diff = (group['TransactionAmount (INR)']-group['CustAccountBalance']).mean()
mean_diff = pd.DataFrame( mean_balance.mean_balance.sub(mean_trans.mean_trans, axis = 0) )
mean_diff.columns = ['mean_diff']

In [633]:
# gender
gender_most_freq = data[['CustomerID', 'CustGender']].groupby(by = 'CustomerID').agg(lambda x:x.value_counts().index[0])
gender_most_freq.rename(columns = {'CustGender' :'gender_most_freq'}, inplace = True)

In [609]:
# most_freq_loc
most_freq_loc = data[['CustomerID','CustLocation']].groupby(by='CustomerID').agg(lambda x: x.value_counts().index[0])
most_freq_loc.rename(columns = {'CustLocation' :'most_freq_loc'}, inplace = True)

In [669]:
dfs = [nr_trans, nr_trans_100p, mean_trans, mean_balance,mean_diff, gender_most_freq, most_freq_loc]

In [686]:
df_7 = pd.concat([nr_trans, nr_trans_100p, mean_trans, mean_balance,mean_diff, gender_most_freq, most_freq_loc], 1)

/var/folders/c0/1q90pl8n3k50gnjpfmzf_lzh0000gn/T/ipykernel_24066/2719202539.py:1: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  df_7 = pd.concat([nr_trans, nr_trans_100p, mean_trans, mean_balance,mean_diff, gender_most_freq, most_freq_loc], 1)


In [688]:
df_7.to_csv('seven_params.csv', sep=',', encoding='utf-8', index = False)

In [ ]:
'TransactionID', 'CustomerID', 'CustomerDOB', 'CustGender',
'CustLocation', 'CustAccountBalance', 'TransactionDate',
'TransactionTime', 'TransactionAmount (INR)'


## 20 features
grouping by CustomerID then working on groups, 
stored in twenty_params.csv

In [762]:
groups = data.groupby(by = 'CustomerID')

In [776]:
from tqdm import tqdm

In [779]:
for i, el in tqdm( enumerate(groups) ): # the i is only here because it had to restart and didn't manage to slice groups
    if i>340000:
        name, group = el
        features_20(name, group)

839081it [38:04, 367.36it/s]  


In [759]:
feat_20 = {}
def features_20(name, group):

    min_trans = group['TransactionAmount (INR)'].min()
    max_trans = group['TransactionAmount (INR)'].max()
    #std_trans = group['TransactionAmount (INR)'].std()
    quant_75_trans = group['TransactionAmount (INR)'].quantile(0.75) #?
    quant_25_trans = group['TransactionAmount (INR)'].quantile(0.25) #?
    trans_amount = group['TransactionAmount (INR)']
    trans_val_nr = trans_amount.nunique() / len(trans_amount)
    
    
    max_balance = group['CustAccountBalance'].max()
    min_balance = group['CustAccountBalance'].min()
    #std_balance = group['CustAccountBalance'].std()
    diff_balance = max_balance - min_balance

    
    times = group['TransactionTime']
    hours = pd.to_datetime(times.astype(str)).dt.hour
    # most_freq_hour = max(set(hours), key=hours.count)
    most_freq_hour = max(set(hours), key=list(hours).count)
    max_time = hours.max()
    min_time = hours.min()
    mean_time = hours.mean()
    
    date = pd.to_datetime(group.TransactionDate.astype(str)).dt
    most_freq_weekday = max(set(date.weekday), key=list(date.weekday).count )
    most_freq_month = max(set(date.month), key=list(date.month).count)
    nr_diff_months = len( set(date.month) )
    
    diff_locs = group['CustLocation'].nunique()
    
    diff_dob = group['CustomerDOB'].nunique() # ? different people
    age = group.TransactionDate.sub(group.CustomerDOB, axis = 0).mean().days/365
    age_mean = int(age)
    
    trans_amount = group['TransactionAmount (INR)']
    nr_trans_1kp = trans_amount[trans_amount>1000].count()
    
    balance_amount = group['CustAccountBalance']
    nr_trans_10kp = balance_amount[balance_amount>10000].count()
    
    trans_balance_ratio = group['TransactionAmount (INR)']/group['CustAccountBalance']
    tb_ratio_mean = trans_balance_ratio.mean()
    tb_ratio_max = trans_balance_ratio.max()


    #avg_nr_trans_perday = group.groupby(by= 'TransactionDate').count().mean()
    #first_freq_loc = group['CustLocation'].agg(lambda x:x.value_counts().index[0])
    
    feat_20[name] = [min_trans, max_trans, quant_75_trans, quant_25_trans, trans_val_nr, max_balance, min_balance, diff_balance, most_freq_hour, max_time, min_time, mean_time, most_freq_weekday, most_freq_month, nr_diff_months, diff_locs, diff_dob, age_mean, nr_trans_1kp, nr_trans_10kp, tb_ratio_mean, tb_ratio_max]
    
    

In [781]:
len(feat_20) # nr groups

839081

In [801]:
data_20 = pd.DataFrame.from_dict(feat_20, orient = 'index')

In [802]:
col_list = 'CustomerID, min_trans, max_trans, quant_75_trans, quant_25_trans, trans_val_nr, max_balance, min_balance, diff_balance, most_freq_hour, max_time, min_time, mean_time, most_freq_weekday, most_freq_month, nr_diff_months, diff_locs, diff_dob, age_mean, nr_trans_1kp, nr_trans_10kp, tb_ratio_mean, tb_ratio_max'
col_list_str = col_list.split(', ')

In [803]:
data_20

,0,1,2,3,4,5,6,7,8,9,...,12,13,14,15,16,17,18,19,20,21
C1021343,163.8,467.0,391.2,239.6,1.0,134698.04,5532.00,129166.04,16,19,...,2,1,2,2,2,31,0,1,0.016538,0.029610
C1040842,1.0,1.0,1.0,1.0,1.0,26387.17,26387.17,0.00,8,8,...,4,4,1,1,1,24,0,1,0.000038,0.000038
C1130989,60.0,60.0,60.0,60.0,1.0,250.26,250.26,0.00,10,10,...,6,5,1,1,1,25,0,0,0.239751,0.239751
C1219943,1892.0,1892.0,1892.0,1892.0,1.0,2951.10,2951.10,0.00,12,12,...,1,3,1,1,1,24,1,0,0.641117,0.641117
C1220223,148.0,148.0,148.0,148.0,1.0,95075.54,95075.54,0.00,17,17,...,0,2,1,1,1,34,0,1,0.001557,0.001557
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
C9099836,691.0,691.0,691.0,691.0,1.0,133067.23,133067.23,0.00,0,0,...,4,7,1,1,1,25,0,1,0.005193,0.005193
C9099877,222.0,222.0,222.0,222.0,1.0,96063.46,96063.46,0.00,12,12,...,3,9,1,1,1,20,0,1,0.002311,0.002311
C9099919,126.0,126.0,126.0,126.0,1.0,5559.75,5559.75,0.00,12,12,...,3,8,1,1,1,22,0,0,0.022663,0.022663
C9099941,50.0,50.0,50.0,50.0,1.0,35295.92,35295.92,0.00,21,21,...,6,8,1,1,1,21,0,1,0.001417,0.001417


In [804]:
data_20 = data_20.reset_index()
data_20.columns = col_list_str
data_20

,CustomerID,min_trans,max_trans,quant_75_trans,quant_25_trans,trans_val_nr,max_balance,min_balance,diff_balance,most_freq_hour,...,most_freq_weekday,most_freq_month,nr_diff_months,diff_locs,diff_dob,age_mean,nr_trans_1kp,nr_trans_10kp,tb_ratio_mean,tb_ratio_max
0,C1021343,163.8,467.0,391.2,239.6,1.0,134698.04,5532.00,129166.04,16,...,2,1,2,2,2,31,0,1,0.016538,0.029610
1,C1040842,1.0,1.0,1.0,1.0,1.0,26387.17,26387.17,0.00,8,...,4,4,1,1,1,24,0,1,0.000038,0.000038
2,C1130989,60.0,60.0,60.0,60.0,1.0,250.26,250.26,0.00,10,...,6,5,1,1,1,25,0,0,0.239751,0.239751
3,C1219943,1892.0,1892.0,1892.0,1892.0,1.0,2951.10,2951.10,0.00,12,...,1,3,1,1,1,24,1,0,0.641117,0.641117
4,C1220223,148.0,148.0,148.0,148.0,1.0,95075.54,95075.54,0.00,17,...,0,2,1,1,1,34,0,1,0.001557,0.001557
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
839076,C9099836,691.0,691.0,691.0,691.0,1.0,133067.23,133067.23,0.00,0,...,4,7,1,1,1,25,0,1,0.005193,0.005193
839077,C9099877,222.0,222.0,222.0,222.0,1.0,96063.46,96063.46,0.00,12,...,3,9,1,1,1,20,0,1,0.002311,0.002311
839078,C9099919,126.0,126.0,126.0,126.0,1.0,5559.75,5559.75,0.00,12,...,3,8,1,1,1,22,0,0,0.022663,0.022663
839079,C9099941,50.0,50.0,50.0,50.0,1.0,35295.92,35295.92,0.00,21,...,6,8,1,1,1,21,0,1,0.001417,0.001417


In [805]:
data_20.to_csv('twenty_params.csv', sep = ',', encoding = 'utf-8')

# dim reduction